In [ ]:
from typing import TypedDict, List, Dict, Optional, Literal, Union
from typing_extensions import TypeAlias, NotRequired
from pydantic import BaseModel, Field
from datetime import datetime
from enum import Enum

class Environment(str, Enum):
    DEV = "dev"
    UAT = "uat"
    PROD = "prod"

class PromptType(str, Enum):
    ECT = "ECT"
    CUSTOMER = "CUSTOMER"
    SUPPLIER = "SUPPLIER"

class EarningsMetadata(BaseModel):
    quarter_s: str
    year_s: str
    company_s: str
    doc_type_s: str
    tags_s: List[str]
    event_time_s: str
    companyId_s: str
    tickers_s: List[str]
    cusips_s: List[str]
    isins_s: List[str]
    sedols_s: List[str]
    document_date_s: str
    objecturl: Optional[str] = None

    class Config:
        frozen = True

class AnalysisSection(BaseModel):
    key_points: List[str]
    summary: str

    class Config:
        frozen = True

class PromptRequest(TypedDict):
    prompt: str
    company: str
    type: PromptType
    bucket: int
    model: str

class PromptResponse(TypedDict):
    result: Dict[str, AnalysisSection]
    type: PromptType
    heading: str
    confidence: float
    bucket: int

class MetadataResponse(TypedDict):
    company: str
    period: str
    date: str
    ticker: NotRequired[str]

class ECTSummaryResponse(TypedDict):
    metadata: MetadataResponse
    analysis: Dict[str, AnalysisSection]

ErrorResponse: TypeAlias = Dict[Literal["error"], str]
ECTResponse: TypeAlias = Union[List[ECTSummaryResponse], List[ErrorResponse]]

In [ ]:
class ECTAnalyzer:
    def __init__(
        self, 
        gssso_token: str, 
        config: Optional[ECTConfig] = None,
        environment: Optional[str] = None
    ):
        self.gssso_token = gssso_token
        self.config = config or get_config(environment)
        self.llm = ConverseAILangchainLLM.from_defaults(
            app_id="fluentai",
            model_name=self.config.model.name
        )
        ect_logger.setLevel(self.config.log_level)

    @log_errors(ect_logger)
    async def get_earnings_by_company_id(self, company_id: str) -> List[EarningsMetadata]:
        ect_logger.info(f"Fetching earnings for company: {company_id}")
        
        async with aiohttp.ClientSession() as session:
            async with session.get(
                self._build_url(company_id),
                headers=self._get_headers(),
                timeout=self.config.api.timeout
            ) as response:
                if response.status != 200:
                    raise APIError(
                        f"Failed to fetch earnings data", 
                        status_code=response.status
                    )
                    
                content = await response.text()
                documents = self._parse_documents(content)
                
                if not documents:
                    raise DocumentNotFoundError(f"No documents found for company {company_id}")
                    
                ect_logger.info(f"Found {len(documents)} documents")
                return documents

    def generate_ect_prompts(
        self, 
        ect: str, 
        id_bb_company: str, 
        html_prompt: str = ""
    ) -> List[PromptRequest]:
        items = [(k, v) for k, v in self.config.headings]
        prompt_buckets = [
            items[i:i + self.config.model.batch_size]
            for i in range(0, len(items), self.config.model.batch_size)
        ]
        
        return [
            {
                "prompt": self._format_bucket_prompt(bucket, ect, html_prompt),
                "company": id_bb_company,
                "type": PromptType.ECT,
                "bucket": bucket_id,
                "model": self.config.model.name
            }
            for bucket_id, bucket in enumerate(prompt_buckets)
        ]

    async def execute_prompts(self, prompts: List[PromptRequest]) -> List[PromptResponse]:
        responses: List[PromptResponse] = []
        for prompt in prompts:
            try:
                response = JsonFormer(
                    schema=AnalysisOutput,
                    llm=self.llm
                ).invoke(prompt["prompt"])
                
                responses.append({
                    "result": response.dict(),
                    "type": prompt["type"],
                    "heading": prompt.get("heading", ""),
                    "confidence": response.confidence,
                    "bucket": prompt.get("bucket", 0),
                })
                
            except Exception as e:
                raise PromptExecutionError(f"Failed to execute prompt: {str(e)}")
                
        return responses

    async def get_ect_summary(self, company_id: str) -> ECTResponse:
        try:
            documents = await self.get_earnings_by_company_id(company_id)
            latest_doc = max(
                documents,
                key=lambda x: datetime.strptime(x.document_date_s, "%Y-%m-%d")
            )
            
            content = await self.get_earnings_content(latest_doc)
            if not content:
                return [{"error": "Could not fetch earnings content"}]
            
            prompts = self.generate_ect_prompts(content, company_id)
            responses = await self.execute_prompts(prompts)
            
            analysis_sections = {
                resp["heading"]: resp["result"]
                for resp in responses
                if resp["confidence"] > self.config.model.confidence_threshold
            }
            
            if not analysis_sections:
                return [{"error": "No high-confidence analysis generated"}]
            
            return [{
                "metadata": {
                    "company": latest_doc.company_s,
                    "period": f"Q{latest_doc.quarter_s} {latest_doc.year_s}",
                    "date": latest_doc.document_date_s,
                    "ticker": latest_doc.tickers_s[0] if latest_doc.tickers_s else None
                },
                "analysis": analysis_sections
            }]
            
        except ECTError as e:
            return [{"error": str(e)}]